In [91]:
## requires: pytorch, transformer, flash-attn
from transformers import AutoModel, AutoTokenizer
import torch
import torch.nn.functional as F
from torch.utils.cpp_extension import CUDA_HOME
import pandas as pd
import numpy as np

In [2]:
CUDA_HOME

'C:\\Program Files\\NVIDIA GPU Computing Toolkit\\CUDA\\v12.6'

In [3]:
model_name = "openbmb/MiniCPM-Embedding"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name, 
                                  trust_remote_code=True, 
                                  attn_implementation="flash_attention_2", 
                                  torch_dtype=torch.float16).to("cuda")
model.eval()

Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.36it/s]


MiniCPMModel(
  (embed_tokens): Embedding(122753, 2304)
  (layers): ModuleList(
    (0-39): 40 x MiniCPMDecoderLayer(
      (self_attn): MiniCPMFlashAttention2(
        (q_proj): Linear(in_features=2304, out_features=2304, bias=False)
        (k_proj): Linear(in_features=2304, out_features=2304, bias=False)
        (v_proj): Linear(in_features=2304, out_features=2304, bias=False)
        (o_proj): Linear(in_features=2304, out_features=2304, bias=False)
        (rotary_emb): MiniCPMRotaryEmbedding()
      )
      (mlp): MiniCPMMLP(
        (gate_proj): Linear(in_features=2304, out_features=5760, bias=False)
        (up_proj): Linear(in_features=2304, out_features=5760, bias=False)
        (down_proj): Linear(in_features=5760, out_features=2304, bias=False)
        (act_fn): SiLU()
      )
      (input_layernorm): MiniCPMRMSNorm()
      (post_attention_layernorm): MiniCPMRMSNorm()
    )
  )
  (norm): MiniCPMRMSNorm()
)

In [4]:
def weighted_mean_pooling(hidden, attention_mask):
    attention_mask_ = attention_mask * attention_mask.cumsum(dim=1)
    s = torch.sum(hidden * attention_mask_.unsqueeze(-1).float(), dim=1)
    d = attention_mask_.sum(dim=1, keepdim=True).float()
    reps = s / d
    return reps

@torch.no_grad()
def encode(input_texts):
    batch_dict = tokenizer(input_texts, max_length=512, padding=True, truncation=True, return_tensors='pt', return_attention_mask=True).to("cuda")
    
    outputs = model(**batch_dict)
    attention_mask = batch_dict["attention_mask"]
    hidden = outputs.last_hidden_state

    reps = weighted_mean_pooling(hidden, attention_mask)   
    embeddings = F.normalize(reps, p=2, dim=1).detach().cpu().numpy()
    return embeddings

In [5]:
queries = ["中国的首都是哪里？"]
passages = ["beijing", "shanghai"]


INSTRUCTION = "Query: "
queries = [INSTRUCTION + query for query in queries]

embeddings_query = encode(queries)
embeddings_doc = encode(passages)

scores = (embeddings_query @ embeddings_doc.T)
print(scores.tolist())  # [[0.3535913825035095, 0.18596848845481873]]

[[0.35365185141563416, 0.18593288958072662]]


In [19]:
df = pd.read_csv("../story_crawler/stories.csv")
df

,title,text,category,story_url
0,梦幻童话《橙子国王与巧克力屋》,来源：中国儿童文学网 作者：小豆儿\n 有一片美丽的森林，那里鲜花明媚，鸟语花香，住着许...,童话故事,http://www.wpwx.cn/news/tonghua/22318144942K96...
1,羊妈妈和她的好心邻居们,来源：中国儿童文学网 作者：王绪化\n 羊妈妈快要生小羊羔了，这一下子不但乐坏了羊妈妈，...,童话故事,http://www.wpwx.cn/news/tonghua/211118135030JB...
2,小鸟开花店,来源：中国儿童文学网 作者：陈彦旭\n江苏省盐城市大丰区城东实验小学文学社读书班 陈彦旭\...,童话故事,http://www.wpwx.cn/news/tonghua/211118122043D8...
3,小猫和公鸡,来源：中国儿童文学网 作者：唐孖欣\n江苏省盐城市大丰区城东实验小学文学社读书班 唐孖欣\...,童话故事,http://www.wpwx.cn/news/tonghua/21111812191118...
4,小白兔和小青蛙,来源：中国儿童文学网 作者：吕金凇\n江苏省盐城市大丰区城东实验小学小海星文学社读书班 吕...,童话故事,http://www.wpwx.cn/news/tonghua/2111181217327C...
...,...,...,...,...
1113,女娲补天,来源：中国民间故事网 作者：佚名\n 有一天，大龙和精卫、小太极一起到远古时代去玩，居然...,神话故事,http://www.wpwx.cn/news/shenhua/07102010574F0H...
1114,白氏郎,来源：中国民间故事网 作者：佚名\n\n泰山周围有吕洞宾三戏白牡丹的传说，据说他们还生了个...,神话故事,http://www.wpwx.cn/news/shenhua/071020105620DJ...
1115,鲤鱼跳龙门,来源：中国民间故事网 作者：佚名\n 庙峡，又名妙峡。两座巍峨雄奇的凤凰大山，拔水擎...,神话故事,http://www.wpwx.cn/news/shenhua/0741319383192K...
1116,盘古开天辟地,来源：中国民间故事网 作者：佚名\n在遥远的太古时代，宇宙好像一颗硕大无比的鸡蛋，里面漆黑...,神话故事,http://www.wpwx.cn/news/shenhua/07413193716DJ9...


In [22]:
columns = ['title', 'text', 'category']
df.loc[:, 'combined'] = df.apply(lambda x: '|'.join(f"{col}: {str(x[col]).strip()}" for col in columns if pd.notna(x[col])), axis=1)

df

,title,text,category,story_url,combined
0,梦幻童话《橙子国王与巧克力屋》,来源：中国儿童文学网 作者：小豆儿\n 有一片美丽的森林，那里鲜花明媚，鸟语花香，住着许...,童话故事,http://www.wpwx.cn/news/tonghua/22318144942K96...,title: 梦幻童话《橙子国王与巧克力屋》|text: 来源：中国儿童文学网 作者：小豆...
1,羊妈妈和她的好心邻居们,来源：中国儿童文学网 作者：王绪化\n 羊妈妈快要生小羊羔了，这一下子不但乐坏了羊妈妈，...,童话故事,http://www.wpwx.cn/news/tonghua/211118135030JB...,title: 羊妈妈和她的好心邻居们|text: 来源：中国儿童文学网 作者：王绪化\n ...
2,小鸟开花店,来源：中国儿童文学网 作者：陈彦旭\n江苏省盐城市大丰区城东实验小学文学社读书班 陈彦旭\...,童话故事,http://www.wpwx.cn/news/tonghua/211118122043D8...,title: 小鸟开花店|text: 来源：中国儿童文学网 作者：陈彦旭\n江苏省盐城市大...
3,小猫和公鸡,来源：中国儿童文学网 作者：唐孖欣\n江苏省盐城市大丰区城东实验小学文学社读书班 唐孖欣\...,童话故事,http://www.wpwx.cn/news/tonghua/21111812191118...,title: 小猫和公鸡|text: 来源：中国儿童文学网 作者：唐孖欣\n江苏省盐城市大...
4,小白兔和小青蛙,来源：中国儿童文学网 作者：吕金凇\n江苏省盐城市大丰区城东实验小学小海星文学社读书班 吕...,童话故事,http://www.wpwx.cn/news/tonghua/2111181217327C...,title: 小白兔和小青蛙|text: 来源：中国儿童文学网 作者：吕金凇\n江苏省盐城...
...,...,...,...,...,...
1113,女娲补天,来源：中国民间故事网 作者：佚名\n 有一天，大龙和精卫、小太极一起到远古时代去玩，居然...,神话故事,http://www.wpwx.cn/news/shenhua/07102010574F0H...,title: 女娲补天|text: 来源：中国民间故事网 作者：佚名\n 有一天，大龙和...
1114,白氏郎,来源：中国民间故事网 作者：佚名\n\n泰山周围有吕洞宾三戏白牡丹的传说，据说他们还生了个...,神话故事,http://www.wpwx.cn/news/shenhua/071020105620DJ...,title: 白氏郎|text: 来源：中国民间故事网 作者：佚名\n\n泰山周围有吕洞宾...
1115,鲤鱼跳龙门,来源：中国民间故事网 作者：佚名\n 庙峡，又名妙峡。两座巍峨雄奇的凤凰大山，拔水擎...,神话故事,http://www.wpwx.cn/news/shenhua/0741319383192K...,title: 鲤鱼跳龙门|text: 来源：中国民间故事网 作者：佚名\n 庙峡，又...
1116,盘古开天辟地,来源：中国民间故事网 作者：佚名\n在遥远的太古时代，宇宙好像一颗硕大无比的鸡蛋，里面漆黑...,神话故事,http://www.wpwx.cn/news/shenhua/07413193716DJ9...,title: 盘古开天辟地|text: 来源：中国民间故事网 作者：佚名\n在遥远的太古时...


In [66]:
encode(df.combined[i])[0].tolist()

[0.013131245039403439,
 -0.00983579270541668,
 0.0010496577015146613,
 -0.016299737617373466,
 -0.0064000231213867664,
 0.016749760136008263,
 0.002571913180872798,
 0.02745177410542965,
 0.026494363322854042,
 -0.012750785797834396,
 0.01289380993694067,
 -0.029246898368000984,
 0.0006150732515379786,
 0.0025967766996473074,
 0.012380681000649929,
 0.0007692397339269519,
 0.02403063327074051,
 0.03650648891925812,
 0.0031965908128768206,
 0.023962048813700676,
 -0.015588434413075447,
 0.00921446643769741,
 -0.002454867586493492,
 0.006409456953406334,
 -0.001896725152619183,
 0.011935721151530743,
 0.016402993351221085,
 0.020818833261728287,
 -0.008495507761836052,
 -0.0001188252936117351,
 0.027165988460183144,
 0.00011610717774601653,
 0.03866222873330116,
 0.020265014842152596,
 0.03896275535225868,
 0.018083160743117332,
 0.013919885270297527,
 0.00131255853921175,
 -0.010128631256520748,
 -0.001383720780722797,
 0.0015017071273177862,
 -0.020957153290510178,
 0.01434006635099649

In [126]:
df.loc[:, 'embedding'] = None
embeddings = []
for i in range(10):
    if i % 10 == 0:
        print(f"Processing row {i}")
    embeddings.extend(encode(df.combined[i]))

0       [0.013131245, -0.009835793, 0.0010496577, -0.0...
1       [-0.010121721, -0.02023252, -0.0016462349, 0.0...
2       [-0.0058293347, -0.029841086, 0.013604202, -0....
3       [-0.0037652855, 0.037260298, 0.0073645287, 0.0...
4       [0.03621632, -0.018420111, 0.02681117, 0.00236...
                              ...                        
1113                                                 None
1114                                                 None
1115                                                 None
1116                                                 None
1117                                                 None
Name: embedding, Length: 1118, dtype: object

In [131]:
df['embedding']

0       None
1       None
2       None
3       None
4       None
        ... 
1113    None
1114    None
1115    None
1116    None
1117    None
Name: embedding, Length: 1118, dtype: object

In [133]:
for i, embedding in enumerate(embeddings):
    print(i)
    df.at[i, 'embedding'] = embedding

0
1
2
3
4
5
6
7
8
9


In [136]:
df.embedding[0][12]

0.00061507325

In [146]:
df_embedding = pd.read_csv("stories_embeddings.csv")
df_embedding.embedding[0]

'[0.013131245039403439, -0.00983579270541668, 0.0010496577015146613, -0.016299737617373466, -0.0064000231213867664, 0.016749760136008263, 0.002571913180872798, 0.02745177410542965, 0.026494363322854042, -0.012750785797834396, 0.01289380993694067, -0.029246898368000984, 0.0006150732515379786, 0.0025967766996473074, 0.012380681000649929, 0.0007692397339269519, 0.02403063327074051, 0.03650648891925812, 0.0031965908128768206, 0.023962048813700676, -0.015588434413075447, 0.00921446643769741, -0.002454867586493492, 0.006409456953406334, -0.001896725152619183, 0.011935721151530743, 0.016402993351221085, 0.020818833261728287, -0.008495507761836052, -0.0001188252936117351, 0.027165988460183144, 0.00011610717774601653, 0.03866222873330116, 0.020265014842152596, 0.03896275535225868, 0.018083160743117332, 0.013919885270297527, 0.00131255853921175, -0.010128631256520748, -0.001383720780722797, 0.0015017071273177862, -0.020957153290510178, 0.014340066350996494, -0.01397117879241705, 0.02633744478225

In [147]:
print(len(embeddings))
print(df.shape[0])

10
1118
